In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys; sys.path.append('..')
from model.began import Generator128
from utils import load_trained_net
from model.vae import VAE
from model.biggan import BigGanSkip
from model.dcgan import Generator as dcgan_generator

In [5]:
from model.deep_decoder import DeepDecoder

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def print_params_info(model):
    print('#'*30)
    print(model)

    g = get_model(model)
    print('idx','latent_dim','percent_weights_cut', 'weights_cut_so_far', 'weights_total')
    num_params_so_far = 0
    num_params_total = sum(p.numel() for p in g.parameters() if p.requires_grad)
    if model == 'biggan':
        num_params_total -= 128000 # We don't use the "embeddings" layer
    for idx, (z1, z2) in enumerate(g.input_shapes[:-1]):
        z2_dim = np.prod(z2) if len(z2) > 0 else 0
        print(idx, np.prod(z1) + z2_dim, f'{num_params_so_far / num_params_total:0.2f}', num_params_so_far, num_params_total)
        if model in ['began', 'vae']:
            num_params_so_far += sum(p.numel() for p in g.layers[idx].parameters() if p.requires_grad)
        elif model == 'dcgan':
            num_params_so_far += sum(p.numel() for p in g.main[idx].parameters() if p.requires_grad)
        elif model == 'biggan':
            if idx == 0:
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.gen_z.parameters() if p.requires_grad)
            elif idx == len(g.input_shapes) - 2:
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.bn.parameters() if p.requires_grad)
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.relu.parameters() if p.requires_grad)
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.conv_to_rgb.parameters() if p.requires_grad)
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.tanh.parameters() if p.requires_grad)
            else:
                num_params_so_far += sum(p.numel() for p in g.biggan.generator.layers[idx-1].parameters() if p.requires_grad)
        else:
            raise

In [ ]:
def get_model(name):
    if model == 'began':
        g = Generator128(64).to('cuda:0')
        g = load_trained_net(g, '../checkpoints/celeba_began.withskips.bs32.cosine.min=0.25.n_cuts=0/gen_ckpt.49.pt')
    elif model == 'vae':
        g = VAE().to('cuda:0')
        g.load_state_dict(torch.load('../vae_checkpoints/vae_bs=128_beta=1.0/epoch_19.pt'))
        g = g.decoder
    elif model == 'biggan':
        g = BigGanSkip().to('cuda:0')
    elif model == 'dcgan':
        g = dcgan_generator().to('cuda:0')  
        g.load_state_dict(torch.load('../dcgan_checkpoints/netG.epoch_24.n_cuts_0.bs_64.b1_0.5.lr_0.0002.pt'))
    else:
        raise
    return g

In [8]:
# Print Deep Decoder parameters
dd_64 = count_parameters(DeepDecoder(num_filters=44, img_size=64, depth=5))
dd_128 = count_parameters(DeepDecoder(num_filters=89, img_size=128, depth=6))
print(dd_64 / (64 * 64 * 3))
print(dd_128 / (128 * 128 * 3))

0.6841634114583334
0.8384195963541666


In [7]:
for model in ['began', 'vae', 'dcgan', 'biggan']:
    print_params_info(model)

NameError: name 'print_params_info' is not defined